In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import sys
import warnings
import os
import configparser
from datetime import timedelta, datetime
sys.path.append("../")
from isri_simulation.isri_simulation import Siminfo, Simulation, Product
from pymoo_utils.algorithm_runner import run_alg_parallel, run_alg
warnings.simplefilter(action='ignore', category=FutureWarning)
from isri_metaheuristic_simple import ISRIProblem, ISRIMutation, ISRICrossover, ISRIDuplicateElimination, ISRISampling, get_random_chromosome, split_jobs_on_lines
pd.set_option('mode.chained_assignment', None)

config_path = 'config.ini'

In [2]:
with open('jobdata_test.pkl', 'rb') as jb_file:
    jobdata = pickle.load(jb_file)
    jobdata = {int(job_id.replace(" ", "")): jobdata[job_id] for job_id in jobdata.keys()}

with open('siminfo_test.pkl', 'rb') as si_file:
    siminfo = pickle.load(si_file)

with open('test_plan.pkl', 'rb') as plan_file:
    test_plan = pickle.load(plan_file)
    
simulation = siminfo
simulation.stations.loc[:, 'currSpeed'] = 190

C:\Users\lukas\AppData\Local\Temp/ipykernel_25364/3963060295.py:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  simulation.stations.loc[:, 'currSpeed'] = 190


In [3]:
# Test Sim
sim = Simulation(simulation)
prod_plan = split_jobs_on_lines(test_plan, 36, lines=2)
simstart = siminfo.start_time
linie_A = [Product(p, jobdata[p]['times'], simstart + timedelta(seconds = jobdata[p]['due_date']), sim=sim) for p in prod_plan[0]]
linie_B = [Product(p, jobdata[p]['times'], simstart + timedelta(seconds = jobdata[p]['due_date']), sim=sim) for p in prod_plan[1]]
sim_plan = {0: linie_A, 1: linie_B}
sim.run(sim_plan, simstart)


In [4]:
el = pd.DataFrame(sim.log.eventlog)
el['MA_Wait'] = el.ReceivedMA - el.Entry
el['WorkTime'] = el.FinishedWork - el.ReceivedMA
el['TransportEnd'] = el.ReachedEnd - el.FinishedWork
el['TotalDuration'] = el.ReachedEnd - el.Entry

In [6]:
# Test for 1000 random Iterations
for _ in range(1000):
    test_plan = get_random_chromosome(36, jobdata)
    print(_)
    with open('test_plan.pkl', 'wb') as outfile:
        pickle.dump(test_plan, outfile)
    sim = Simulation(simulation)
    prod_plan = split_jobs_on_lines(test_plan, 36, lines=2)
    simstart = siminfo.start_time
    linie_A = [Product(p, jobdata[p]['times'], simstart + timedelta(seconds = jobdata[p]['due_date']), sim=sim) for p in prod_plan[0]]
    linie_B = [Product(p, jobdata[p]['times'], simstart + timedelta(seconds = jobdata[p]['due_date']), sim=sim) for p in prod_plan[1]]
    sim_plan = {0: linie_A, 1: linie_B}
    sim.run(sim_plan, simstart)

0


RuntimeError: Linie war 39.92 unterbrochen, es sind aber nur 0.0 Sekunden vergangen

RuntimeError: Linie war 104.44 unterbrochen, es sind aber nur 0.0 Sekunden vergangen

In [24]:
el = pd.DataFrame(sim.log.eventlog)
el['WaitEmployee'] = el.ReceivedMA - el.Entry
el['Duration'] = el.ReachedEnd - el.Entry

In [27]:
el.groupby('Substation').agg('max')

,Product,Station,Entry,ReceivedMA,FinishedWork,ReachedEnd,WaitEmployee,Duration
Substation,,,,,,,,
1,23080798929,LinieB,5822.76,5822.76,5921.22,6034.76,74.88,212.00
2,23080798929,LinieB,6034.76,6034.76,6113.90,6496.54,142.24,541.58
3,23080798929,LinieB,6496.54,6496.54,6569.32,6753.94,94.70,554.06
4,23080798929,LinieB,6753.94,6753.94,6753.94,7043.16,65.94,532.98
5,23080798929,LinieB,7043.16,7043.16,7061.52,7285.42,141.36,541.58
6,23080798929,LinieB,7285.42,7285.42,7311.02,7637.88,127.92,541.58
7,23080798929,LinieB,7637.88,7637.88,7660.82,8043.58,122.04,554.06
8,23080798929,LinieB,8043.58,8043.58,8118.58,8458.42,188.44,541.58
9,23080798929,LinieB,8458.42,8458.42,8509.94,8958.36,148.64,702.78


In [2]:
errors = pd.DataFrame(sim.log.error_log)
errors['Duration'] = errors.End - errors.Start
errors[errors.Station == 'LinieA']

NameError: name 'pd' is not defined

In [3]:
errors.Station.value_counts()

NameError: name 'errors' is not defined

In [7]:
# Test Optimisation
config = configparser.ConfigParser()
config.read(config_path)
n_jobs = len(jobdata)
n_machines = siminfo.n_machines
param = {
        "jobs": jobdata,
        "problem_getter": ISRIProblem,
        "crossover": ISRICrossover(config['GA'].getfloat('cx_rate')),
        "mutation": ISRIMutation(config['GA'].getfloat('mut_rate')),
        "sampling": ISRISampling(),
        "eliminate_duplicates": ISRIDuplicateElimination(),
        "n_jobs": n_jobs,
        "n_machines": n_machines,
        "pop_size": config['GA'].getint('pop_size'),
        "jobs_per_line": config['GA'].getint('jobs_per_line'),
        "alg_name": config['GA']['alg_name'],  # nsga2 | moead | rvea | ctaea | unsga3
        "obj_weights": [config['GA'].getfloat('weight_workload'), config['GA'].getfloat('weight_deadline')],  # 1: min geglättete belastung, max minimalen due date puffer
        "conveyor_speed": config['GA'].getint('conveyor_speed'),
        "ma_window_size": config['GA'].getint('ma_window_size'),
        "n_evals": config['GA'].getint('n_evals'),  # number of fitness calculations,
        "seed": config['GA'].getint('seed'),
        "parallelization_method": 'multi_processing',  # multi_threading | multi_processing | dask
        "n_parallelizations": config['GA'].getint('n_parallelizations'),  # number threads or processes,
        "print_results": config['GA'].getboolean('print_results'),
        "print_iterations": config['GA'].getboolean('print_iterations'),  # number threads or processes,
        "adaptive_rate_mode": config['GA']['adaptive_rate_mode'],  # none | icdm | imdc | rl
        "criterion": config['GA']['criterion'],  # 'minmax' | 'minavg' | 'minvar',
        'plot_solution': config['GA'].getboolean('plot_solutions'),
        'use_sim': config['GA'].getboolean('use_sim'),
        'simstart': siminfo.start_time,
        'simulation': simulation,
        'agent': None, 
        'train_agent': config['GA'].getboolean('train_agent'),
        'reward_func': None
    }

# Optimierung
problem = ISRIProblem(param, runner=None)
res, best_chromosome, best_fitness = run_alg_parallel(run_alg, param)

4.78 s | EVALS: 50 | CXPB: 0.4, MUTPB: 0.6 | MEAN FITNESSES: [35225.565, 64800.0] | MIN FITNESSES: [35225.565, 64800.0]
6.41 s | EVALS: 100 | CXPB: 0.4, MUTPB: 0.6 | MEAN FITNESSES: [35192.4656, 65055.0832] | MIN FITNESSES: [34187.565, 64800.0]
7.82 s | EVALS: 150 | CXPB: 0.4, MUTPB: 0.6 | MEAN FITNESSES: [35162.2724, 65187.76640000001] | MIN FITNESSES: [34187.565, 64800.0]
9.29 s | EVALS: 200 | CXPB: 0.4, MUTPB: 0.6 | MEAN FITNESSES: [34974.8036, 67611.93] | MIN FITNESSES: [33275.985, 64800.0]
10.68 s | EVALS: 250 | CXPB: 0.4, MUTPB: 0.6 | MEAN FITNESSES: [34693.8378, 72154.7644] | MIN FITNESSES: [33046.51500000001, 64800.0]
12.04 s | EVALS: 300 | CXPB: 0.4, MUTPB: 0.6 | MEAN FITNESSES: [34417.6104, 69590.4888] | MIN FITNESSES: [32921.94500000001, 64800.0]
13.48 s | EVALS: 350 | CXPB: 0.4, MUTPB: 0.6 | MEAN FITNESSES: [34166.870200000005, 70361.02160000001] | MIN FITNESSES: [31971.380000000005, 64800.0]
15.12 s | EVALS: 400 | CXPB: 0.4, MUTPB: 0.6 | MEAN FITNESSES: [33890.060900000004